# Precipitation analysis

This notebook is to analze and regress precipitaion data with results from ONI, CANI, and EANI.

# Imports

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import cftime
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import xeofs as xe
import glob
from geocat.viz import util as gvutil
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.util as cutil
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

# PBSClustter

In [ ]:
# Create a PBS cluster object
cluster = PBSCluster(account='P93300313',
                     job_name='ATLN-ENSO-CESMLE2',
                     cores=1,
                     memory='8GiB',
                     processes=1,
                     walltime='02:00:00',
                     queue='casper',
                     interface='ext',
                     n_workers=1)

# dont scale many workers unless using LE
# cluster.scale(10)

client = Client(cluster)
client

In [ ]:
cluster.scale(2)

In [ ]:
# client.shutdown()
cluster.workers

# Useful functions

In [ ]:
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit


def ds_map(ds_to_plt, bounds=[20, -60, 10, -10], name='figure'):
    fig, ax = plt.subplots(1, 1,
                           subplot_kw={'projection': ccrs.PlateCarree()})
    fig.subplots_adjust(hspace=0, wspace=0, top=0.925, left=0.1)
    cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
    cdat, clon = cutil.add_cyclic_point(ds_to_plt, ds_to_plt.longitude)

    ax.set_title(name)
    lat_ticks = np.arange(bounds[3], bounds[2], 5)
    lon_ticks = np.arange(bounds[1], bounds[0], 10)
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cf.LAND)

# Data Imports

In [ ]:
files = glob.glob('/glade/campaign/collections/rda/data/d728008/gpcp_v3.2_monthly/*/*.nc4')
precip = xr.open_mfdataset(files, engine='netcdf4')
precip = precip['sat_gauge_precip']
precip.compute()

# Results Import

# Regrid

In [ ]:
# regrid GPCP data to fit with HadiSST results 1x1

# select data

In [ ]:
# selected dates
dates = xr.date_range(start='1920-01-16', end='2025-01-16', freq='1ME')
ds = ds.sel(time=dates, method='nearest')
# select summer
# summer_ds = ds.where(ds['time'].dt.month.isin([6, 7, 8]), drop=True)

In [ ]:
# select onset MAM
mam_precip = precip.where(precip['time'].dt.month.isin([3, 4, 5]), drop=True)
mam_precip.compute()
# select summer
jja_precip = precip.where(precip['time'].dt.month.isin([6, 7, 8]), drop=True)
jja_precip.compute()
# select SON
son_precip = precip.where(precip['time'].dt.month.isin([6, 7, 8]), drop=True)
son_precip.compute()